# LABORATORIO 2: Optimización de Redes

Jairo Garavito Correa - 202111499

In [2]:
from pyomo.environ import *

from pyomo.opt import SolverFactory

import matplotlib.pyplot as plt

## Problem 1: transport networks

Sets:

* Origin cities: S = {Bogotá, Medellín}
* destination cities: D = {Cali, Barranquilla, Pasto, Tunja, Chía, Manizales}

Parameters:

* Offer of the origin cities:
$$O_s = \forall s \in S$$

* Demand of the destination cities:
$$P_d = \forall d \in D$$

* Cost to send products from a supplier city to a customer city:
$$E_{s,d} = \forall {s \in S, d \in D}$$

decision variables:

* Tons of products to be shipped from each city of origin to each destination city:
$$X_{s,d} = \forall {s \in S, d \in D}$$

objective function:

* Minimize transport cost:
$$\text{Minimize} \sum_{s \in S} \sum_{d \in D} X_{s,d} \times E_{s,d}$$

Constraints:

* Demand of destination cities:

$$\sum_{s \in S}  X_{s,d} \geq P_d \space \forall d \in D $$

* Offer of the origin cities:

$$\sum_{d \in D}  X_{s,d} \leq O_s \space \forall s \in S $$


In [5]:
model = ConcreteModel()

# Conjuntos
model.S = ['Bogotá', 'Medellín']  # Ciudades de origen
model.D = ['Cali', 'Barranquilla', 'Pasto', 'Tunja', 'Chía', 'Manizales']  # Ciudades de destino

# Parámetros
# Oferta en las ciudades de origen
model.O = {'Bogotá': 550, 'Medellín': 700}

# Demanda en las ciudades de destino
model.P = {'Cali': 125, 'Barranquilla': 175, 'Pasto': 225, 'Tunja': 250, 'Chía': 225, 'Manizales': 200}

# Costos de transporte de las ciudades de origen a las de destino
model.E = {
    ('Bogotá', 'Cali'): 100000000,
    ('Bogotá', 'Barranquilla'): 2.5,
    ('Bogotá', 'Pasto'): 1.6,
    ('Bogotá', 'Tunja'): 1.4,
    ('Bogotá', 'Chía'): 0.8,
    ('Bogotá', 'Manizales'): 1.4,
    ('Medellín', 'Cali'): 2.5,
    ('Medellín', 'Barranquilla'): 100000000,
    ('Medellín', 'Pasto'): 2.0,
    ('Medellín', 'Tunja'): 1.0,
    ('Medellín', 'Chía'): 1.0,
    ('Medellín', 'Manizales'): 0.8
}

# Variables de decisión
model.X = Var(model.S, model.D, domain=NonNegativeReals)

# Función objetivo: minimizar el costo de transporte
def objective_rule(model):
    return summation(model.E, model.X)
model.OBJ = Objective(rule=objective_rule, sense=1)  # sense=1 para minimizar

# Restricciones

# Restricción de demanda en las ciudades de destino
def demand_rule(model, d):
    return sum(model.X[s, d] for s in model.S) >= model.P[d]
model.DemandConstraint = Constraint(model.D, rule=demand_rule)

# Restricción de oferta en las ciudades de origen
def offer_rule(model, s):
    return sum(model.X[s, d] for d in model.D) <= model.O[s]
model.OfferConstraint = Constraint(model.S, rule=offer_rule)

# Resolver el modelo
solver = SolverFactory('glpk')
solver.solve(model)

model.display()

Model unknown

  Variables:
    X : Size=12, Index={Bogotá, Medellín}*{Cali, Barranquilla, Pasto, Tunja, Chía, Manizales}
        Key                          : Lower : Value : Upper : Fixed : Stale : Domain
          ('Bogotá', 'Barranquilla') :     0 : 175.0 :  None : False : False : NonNegativeReals
                  ('Bogotá', 'Cali') :     0 :   0.0 :  None : False : False : NonNegativeReals
                  ('Bogotá', 'Chía') :     0 : 150.0 :  None : False : False : NonNegativeReals
             ('Bogotá', 'Manizales') :     0 :   0.0 :  None : False : False : NonNegativeReals
                 ('Bogotá', 'Pasto') :     0 : 225.0 :  None : False : False : NonNegativeReals
                 ('Bogotá', 'Tunja') :     0 :   0.0 :  None : False : False : NonNegativeReals
        ('Medellín', 'Barranquilla') :     0 :   0.0 :  None : False : False : NonNegativeReals
                ('Medellín', 'Cali') :     0 : 125.0 :  None : False : False : NonNegativeReals
                ('Medell